In [ ]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.4 MB/s eta 0:00:00


In [ ]:
import os
import zipfile
import pydicom
import csv
from datetime import datetime
import re

def tinhtuoi(tuoi):

        today = datetime.today()
        tuoi = datetime.strptime(tuoi, '%Y%m%d')
        age = today.year - tuoi.year - ((today.month, today.day) < (tuoi.month, tuoi.day))
        return age
def sex(gender):
    if gender == 'M':
        return 'Nam'
    else:
        return 'Nu'

def tinhten(name):
    name = re.sub(r'\b(M|F|T)\b', '', str(name))
    phanten = re.findall(r'\b[A-Za-zÀ-ỹ]+\b', name)
    return ' '.join(phanten)

def in4(dicom):
    try:
        ds = pydicom.dcmread(dicom, force=True)
        name = ds.PatientName
        patient_birth_date = ds.PatientBirthDate
        patient_sex = ds.PatientSex
        xung = ds.SeriesDescription
        return {
            'PatientName': tinhten(name),
            'Age': tinhtuoi(patient_birth_date),
            'PatientSex': sex(patient_sex),
            'SeriesDescription': xung
        }
    except Exception as e:
        print(f"Error reading {dicom}: {e}")
        return {
            'PatientName': 'Unknown',
            'Age': 'Unknown',
            'PatientSex': 'Unknown',
            'SeriesDescription': 'Unknown'
        }

def giainen(path, extract):
    zip = {}
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.zip'):
                zip_file_path = os.path.join(root, file)
                extract_folder = os.path.join(extract, os.path.splitext(file)[0][:100])
                try:
                    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                        zip_ref.extractall(extract_folder)
                    zip[extract_folder] = file
                except zipfile.BadZipFile:
                    print(f"BadZipFile Error: {zip_file_path} is not a zip file or it is corrupted.")
                except zipfile.LargeZipFile:
                    print(f"LargeZipFile Error: {zip_file_path} requires ZIP64 functionality but it is not enabled.")
                except PermissionError:
                    print(f"PermissionError: Permission denied for {zip_file_path}.")
                except Exception as e:
                    print(f"An unexpected error occurred while extracting {zip_file_path}: {e}")
    return zip

patient_info_list = []

path = '/content/drive/MyDrive/Không bệnh lý'
extracted_folder_path = '/content/extracted_files1'

zip = giainen(path, extracted_folder_path)

for folder, zip_name in zip.items():
    for root, dirs, files in os.walk(folder):
        dicom_files = [file for file in files if file.endswith('.dcm')]
        if dicom_files:
            dicom_file_path = os.path.join(root, dicom_files[0])
            patient_info = in4(dicom_file_path)
            patient_info_list.append({
                'Path': os.path.join(zip_name, root),
                'NumberOfDicomFiles': len(dicom_files),
                'PatientName': patient_info['PatientName'],
                'Age': patient_info['Age'],
                'PatientSex': patient_info['PatientSex'],
                'SeriesDescription': patient_info['SeriesDescription']
            })
csv_file = 'patient_info.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['Path', 'NumberOfDicomFiles', 'PatientName', 'Age', 'PatientSex', 'SeriesDescription'])
    writer.writeheader()
    writer.writerows(patient_info_list)

    from google.colab import files
    files.download(csv_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>